In [1]:
# Import des bibliothèques nécessaires
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

def gbm_multi(n_step=252, n_scenario=1000, mu=None, sigma=None, corr_matrix=None, p_0=100, allocation=None):
    """
    Simuler un portefeuille multi-actifs via un processus de type GBM corrélé (Monte Carlo).

    :param n_step: nombre de pas de temps (par défaut 252 jours ouvrés)
    :param n_scenario: nombre de scénarios de Monte Carlo
    :param mu: array des rendements annuels attendus
    :param sigma: array des volatilités annuelles
    :param corr_matrix: matrice de corrélation entre les actifs
    :param p_0: valeur initiale du portefeuille
    :param allocation: poids alloués à chaque actif dans le portefeuille
    :return: matrice (n_step x n_scenario) représentant la valeur du portefeuille dans le temps
    """
    mu = np.array(mu)
    sigma = np.array(sigma)
    allocation = np.array(allocation)
    corr_matrix = np.array(corr_matrix)
    n_asset = len(mu)

    # Contrôles de cohérence
    assert mu.shape[0] == sigma.shape[0] == allocation.shape[0] == corr_matrix.shape[0] == corr_matrix.shape[1], "Shape mismatch among inputs"
    assert np.isclose(np.sum(allocation), 1), "Allocation weights must sum to 1"
    assert p_0 > 0, "Initial portfolio must be positive"

    # Construction de la matrice de covariance via la décomposition de Cholesky
    cov_matrix = np.outer(sigma, sigma) * corr_matrix
    chol_matrix = np.linalg.cholesky(cov_matrix)

    # Simulation de chocs normalisés corrélés
    Z = np.random.normal(size=(n_step, n_scenario, n_asset))
    correlated_Z = Z @ chol_matrix.T

    # Calcul du drift et de la diffusion
    dt = 1 / 252  # Pas de temps quotidien
    drift = (1 + mu) ** dt
    diffusion = np.sqrt(dt)

    # Simulation des rendements
    returns = np.empty((n_step, n_scenario, n_asset))
    returns[0] = 1
    for i in range(1, n_step):
        returns[i] = drift * np.exp(correlated_Z[i] * diffusion)

    # Agrégation des valeurs du portefeuille selon les allocations
    return (p_0 * allocation * np.cumprod(returns, axis=0)).sum(axis=2)

# Widgets de saisie des paramètres de simulation
simulation_params_box = widgets.VBox([
    widgets.BoundedIntText(value=1000, min=100, max=1000000, step=1, description='Value at t=0'),
    widgets.BoundedIntText(value=252, min=1, max=2520, step=1, description='N steps'),
    widgets.BoundedIntText(value=1000, min=1, max=10000, step=1, description='N scenario')
])

# Widget pour sélectionner le nombre d'actifs
n_asset = widgets.IntSlider(value=3, min=1, max=10, step=1, description='N assets')

# Conteneurs pour les champs d’entrée des actifs
asset_box = widgets.VBox()
asset_widgets_list = []
ptf_dict = {}

def create_or_update_widgets(n_asset):
    """
    Créer ou ajuster dynamiquement les widgets selon le nombre d'actifs sélectionnés.
    """
    current_len = len(asset_widgets_list)
    if n_asset > current_len:
        for i in range(current_len, n_asset):
            ticker_widget = widgets.Text(description=f'Asset {i+1}', placeholder='Ticker')
            weight_widget = widgets.BoundedFloatText(value=0.0, min=0, max=100, step=1, description='Allocation')
            asset_widgets_list.append(widgets.HBox([ticker_widget, weight_widget]))
    elif n_asset < current_len:
        del asset_widgets_list[n_asset:]

    for idx, hbox in enumerate(asset_widgets_list[:n_asset]):
        hbox.children[0].description = f'Asset {idx+1}'
        hbox.children[1].description = 'Allocation'
    asset_box.children = asset_widgets_list[:n_asset]

# Lier le slider de nombre d'actifs à la mise à jour des widgets
widget = widgets.interactive(create_or_update_widgets, n_asset=n_asset)

# Bouton de lancement de la simulation et zone d’affichage
button = widgets.Button(description="Run Simulation", button_style='success')
output = widgets.Output()

def run_simulation(b):
    """
    Exécuter la simulation : téléchargement des données, estimation des paramètres, simulation GBM, et affichage.
    """
    with output:
        clear_output()
        ptf_dict.clear()

        # Récupération des tickers et allocations
        for line in asset_box.children:
            ticker = line.children[0].value.strip()
            allocation = line.children[1].value / 100
            if ticker:
                ptf_dict[ticker] = allocation

        # Vérifier que les poids totalisent 100%
        if not ptf_dict or not np.isclose(sum(ptf_dict.values()), 1):
            print("Error: Allocations must sum to 100%")
            return

        print("Asset allocation:")
        for ticker, allocation in ptf_dict.items():
            print(f"{ticker} : {allocation:.2%}")

        # Téléchargement des prix sur 10 ans
        history = pd.DataFrame()
        for ticker in ptf_dict.keys():
            data = yf.Ticker(ticker).history('10y')['Close']
            data.name = ticker
            history = pd.concat([history, data], axis=1)

        # Prétraitement des données
        history.index = pd.to_datetime(history.index)
        history = history.resample('B').last().ffill()
        history.index = history.index.tz_localize(None)
        history = history.dropna()

        # Calcul des paramètres : rendement, volatilité, corrélation
        mu = np.log(history / history.shift(1)).mean().values * 252
        sigma = np.log(history / history.shift(1)).std().values * np.sqrt(252)
        corr_matrix = history.pct_change(fill_method=None).corr()

        print(f"\nAsset parameters:\n{pd.DataFrame([mu*100, sigma*100], index=['Mu', 'Sigma'], columns=ptf_dict.keys()).T.round(2)}")
        print(f"\nCorrelation matrix:\n{corr_matrix.round(2)}")

        # Lancer la simulation GBM
        simulation_results = gbm_multi(
            n_step=simulation_params_box.children[1].value,
            n_scenario=simulation_params_box.children[2].value,
            mu=mu,
            sigma=sigma,
            corr_matrix=corr_matrix.values,
            p_0=simulation_params_box.children[0].value,
            allocation=list(ptf_dict.values())
        )

        # Calcul des percentiles pour les bandes de confiance
        p10 = np.percentile(simulation_results, 10, axis=1)
        p25 = np.percentile(simulation_results, 25, axis=1)
        p50 = np.percentile(simulation_results, 50, axis=1)
        p75 = np.percentile(simulation_results, 75, axis=1)
        p90 = np.percentile(simulation_results, 90, axis=1)
        mean = np.mean(simulation_results, axis=1)

        # Affichage des résultats
        plt.figure(figsize=(10, 6))
        plt.fill_between(range(simulation_params_box.children[1].value), p10, p25, color="#004c99", alpha=1, label="10e percentile")
        plt.fill_between(range(simulation_params_box.children[1].value), p25, p50, color="#3385ff", alpha=1, label="25e percentile")
        plt.fill_between(range(simulation_params_box.children[1].value), p50, p75, color="#99c2ff", alpha=1, label="Médiane")
        plt.fill_between(range(simulation_params_box.children[1].value), p75, p90, color="#cce0ff", alpha=1, label="75e percentile")
        plt.plot(range(simulation_params_box.children[1].value), mean, color="darkorange", linewidth=2, label="Moyenne")
        plt.legend()
        plt.title("Trajectoire du portefeuille avec intervalles de confiance")
        plt.xlabel("Étapes")
        plt.ylabel("Rendement base 100")
        plt.grid(axis='y', linestyle='--', alpha=0.5)
        plt.tight_layout()
        plt.show()

# Lier le bouton à la fonction de simulation
button.on_click(run_simulation)

# Affichage de tous les éléments interactifs
display(
    widgets.VBox([widgets.Label("Simulation Parameters"), simulation_params_box]),
    widgets.VBox([widgets.Label("Assets Parameters"), n_asset, asset_box]),
    button,
    output
)

Button(button_style='success', description='Run Simulation', style=ButtonStyle())

Output()

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

# === GBM Simulation Function ===
def gbm_multi(n_step, n_scenario, mu, sigma, corr_matrix, p_0, allocation):
    mu, sigma, allocation = map(np.array, (mu, sigma, allocation))
    cov_matrix = np.outer(sigma, sigma) * corr_matrix
    chol_matrix = np.linalg.cholesky(cov_matrix)

    Z = np.random.normal(size=(n_step, n_scenario, len(mu)))
    correlated_Z = Z @ chol_matrix.T

    dt = 1 / 252
    drift = (1 + mu) ** dt
    diffusion = np.sqrt(dt)

    returns = np.empty_like(correlated_Z)
    returns[0] = 1
    returns[1:] = drift * np.exp(correlated_Z[1:] * diffusion)

    portfolio = p_0 * allocation * np.cumprod(returns, axis=0)
    return portfolio.sum(axis=2)

# === Widgets: Paramètres de simulation ===
params_dict = {
    'p_0': widgets.BoundedIntText(value=1000, min=100, max=1000000, description='Value at t=0'),
    'n_step': widgets.BoundedIntText(value=252, min=1, max=2520, description='N steps'),
    'n_scenario': widgets.BoundedIntText(value=1000, min=1, max=10000, description='N scenarios')
}
simulation_params_box = widgets.VBox(list(params_dict.values()))

# === Widgets: Nombre d'actifs et leurs caractéristiques ===
n_asset_slider = widgets.IntSlider(value=3, min=1, max=10, description='N assets')
asset_widgets_list = []
asset_box = widgets.VBox()

def update_asset_widgets(n):
    while len(asset_widgets_list) < n:
        asset_widgets_list.append(
            widgets.HBox([
                widgets.Text(placeholder='Ticker'),
                widgets.BoundedFloatText(value=0.0, min=0, max=100, step=1)
            ])
        )
    asset_box.children = asset_widgets_list[:n]

widgets.interactive(update_asset_widgets, n=n_asset_slider)

# === Fonction principale ===
output = widgets.Output()

def run_simulation(_):
    with output:
        clear_output()
        tickers, weights = [], []

        for row in asset_box.children:
            ticker = row.children[0].value.strip().upper()
            weight = row.children[1].value / 100
            if ticker:
                tickers.append(ticker)
                weights.append(weight)

        if not tickers or not np.isclose(sum(weights), 1):
            print("Erreur : les allocations doivent totaliser 100%")
            return

        print("Allocations :")
        for t, w in zip(tickers, weights):
            print(f"{t}: {w:.2%}")

        try:
            data = yf.download(tickers, period='10y', auto_adjust=True)['Close']
            if isinstance(data, pd.Series):  # Cas d'un seul actif
                data = data.to_frame()
            data = data.resample('B').last().ffill().dropna()
        except Exception as e:
            print(f"Erreur lors du téléchargement : {e}")
            return

        log_returns = np.log(data / data.shift(1)).dropna()
        mu = log_returns.mean().values * 252
        sigma = log_returns.std().values * np.sqrt(252)
        corr = log_returns.corr().values

        print("\nParamètres des actifs :")
        print(pd.DataFrame({'Mu': mu * 100, 'Sigma': sigma * 100}, index=tickers).round(2))
        print("\nMatrice de corrélation :")
        print(pd.DataFrame(corr, index=tickers, columns=tickers).round(2))

        sim = gbm_multi(
            n_step=params_dict['n_step'].value,
            n_scenario=params_dict['n_scenario'].value,
            mu=mu,
            sigma=sigma,
            corr_matrix=corr,
            p_0=params_dict['p_0'].value,
            allocation=weights
        )

        percentiles = {
            'p10': np.percentile(sim, 10, axis=1),
            'p25': np.percentile(sim, 25, axis=1),
            'p50': np.percentile(sim, 50, axis=1),
            'p75': np.percentile(sim, 75, axis=1),
            'p90': np.percentile(sim, 90, axis=1),
            'mean': np.mean(sim, axis=1)
        }

        steps = range(params_dict['n_step'].value)
        plt.figure(figsize=(10, 6))
        plt.fill_between(steps, percentiles['p10'], percentiles['p25'], color="#004c99", label="10-25e")
        plt.fill_between(steps, percentiles['p25'], percentiles['p50'], color="#3385ff", label="25-50e")
        plt.fill_between(steps, percentiles['p50'], percentiles['p75'], color="#99c2ff", label="50-75e")
        plt.fill_between(steps, percentiles['p75'], percentiles['p90'], color="#cce0ff", label="75-90e")
        plt.plot(steps, percentiles['mean'], color="darkorange", label="Moyenne", linewidth=2)
        plt.title("Simulation Monte Carlo du portefeuille")
        plt.xlabel("Jours ouvrés")
        plt.ylabel("Valeur simulée")
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.legend()
        plt.tight_layout()
        plt.show()

# === Interface ===
button = widgets.Button(description="Run Simulation", button_style='success')
button.on_click(run_simulation)

display(
    widgets.VBox([
        widgets.Label("Paramètres de simulation"), simulation_params_box,
        widgets.Label("Nombre d'actifs"), n_asset_slider,
        asset_box, button, output
    ])
)


[                       0%                       ]

[                       0%                       ]

[**********************90%******************     ]  9 of 10 completed